In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
options = Options()
#options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")
options.add_experimental_option("prefs", {
  "download.default_directory": "D\\Python\\DataForge-automated\\MathTHPT2025",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": False,
  "safebrowsing.ebabled": "false"
})
driver = webdriver.Chrome(options=options)
useOutsideOCR = 1
driver.get("https://dotsocr.xiaohongshu.com/" if useOutsideOCR == 0 else "https://deepseek-ocr.trunghsgs.edu.vn/gradio/")

try:
    wait = WebDriverWait(driver, 60)
except NameError:
    wait = WebDriverWait(driver, 60)

try:
    wait.until(lambda d: d.execute_script("return document.readyState") == "complete")

    try:
        wait.until(EC.presence_of_element_located((By.ID, "parse_button")))
    except Exception:
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='file']")))
        except Exception:
            pass
    time.sleep(1) 
    print("Page loaded")
except Exception as e:
    print("Timeout waiting for page to load:", e)

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os, json, re

env_path = find_dotenv()
if env_path:
    load_dotenv(env_path)
    print(f"Loaded .env from: {env_path}")
else:
    load_dotenv()
    print("No .env file found; loaded environment from system variables")
    
data_dir = os.environ.get("DATA_DIR", "MathTHPT2025")
loading_folder_env = os.environ.get("LOADING_FOLDER", '["1.Chuyen de", "2.De chac diem 8", "3.De chac diem 9", "4.De luyen them", "5.De quan trong", "6.De so"]')
try:
    loading_folder = json.loads(loading_folder_env)
except Exception:
    try:
        loading_folder = ast.literal_eval(loading_folder_env)
    except Exception:
        loading_folder = [s.strip() for s in re.split(r'[;,]', loading_folder_env) if s.strip()]

In [ ]:
loading_folder

In [ ]:
import os
from pathlib import Path
import logging
from logging.handlers import RotatingFileHandler

for folder in loading_folder:
    pdf_files = []
    folder_path = os.path.join(data_dir, folder)
    for file in os.listdir(folder_path):
        if file.lower().endswith('.pdf'): #Duyet tung file pdf
            pdf_files.append(file)
            file_path = os.path.abspath(os.path.join(folder_path, file))

            #upload file
            file_input = driver.find_element(By.CSS_SELECTOR, "input[type='file'][data-testid='file-upload']")
            file_input.send_keys(file_path)

            print("Sent file to upload input:", file_path)

            #doi upload xong thi click nut parse
            upload_selector = "span.uploading"
            timeout1 = 1800
            try:
                upload_wait = WebDriverWait(driver, timeout1)
                upload_wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, upload_selector)))
                print("Upload indicator disappeared")
            except Exception as e:
                print("Timeout waiting for upload indicator to disappear:", e)
            time.sleep(2)
            wait = WebDriverWait(driver, 30)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button.icon-button[aria-label="Fullscreen"]')))
            parse_btn = driver.find_element(By.ID, "component-15")
            parse_btn.click()
            print("Clicked parse button")

            file_no_ext = os.path.splitext(file)[0]

            download_dir = os.path.abspath(
                os.path.join(folder_path, file_no_ext, "output")
            )

            os.makedirs(download_dir, exist_ok=True)

            before_files = set(os.listdir(download_dir))

            try:
                driver.execute_cdp_cmd(
                    "Page.setDownloadBehavior",
                    {
                        "behavior": "allow",
                        "downloadPath": download_dir
                    }
                )
                print("Download dir set to:", download_dir)
            except Exception as e:
                print("Could not set download directory via CDP:", e)

            wait = WebDriverWait(driver, 7200)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#component-22 div.wrap.default.full.svelte-btia7y.hide")))
            print("Processing completed, element class changed.")
            btn = wait.until(EC.element_to_be_clickable((By.ID, "component-19")))
            print("Target download dir:", os.path.join(os.getcwd(), download_dir))
            print("Download button found, clicking...")
            btn.click()
            time.sleep(30)
            btn.click()

            #doi download xong
            timeout = 1800
            end_time = time.time() + timeout
            downloaded = None
            while time.time() < end_time:
                time.sleep(0.5)
                new_files = set(os.listdir(download_dir)) - before_files
                if new_files:
                    for f in new_files:
                        if not f.endswith('.crdownload'):
                            downloaded = f
                            break
                if downloaded:
                    break
                
            if downloaded:
                print("Downloaded file:", downloaded)
                src = os.path.abspath(os.path.join(download_dir, downloaded))
                dst = os.path.abspath(os.path.join(download_dir, "final.md"))
                try:
                    if not os.path.exists(src):
                        raise FileNotFoundError(f"Source not found: {src}")
                    if os.path.exists(dst):
                        os.remove(dst)
                    os.replace(src, dst)
                    downloaded = os.path.basename(dst)
                    print("Renamed downloaded file to:", dst)
                except Exception as e:
                    print("Failed to rename downloaded file:", e)
            else:
                print("No completed download detected within timeout.")

            clear_btn = driver.find_element(By.CSS_SELECTOR, "button.icon-button[aria-label='Clear']")
            clear_btn.click()
            time.sleep(10)
    print(pdf_files)

In [ ]:
print(download_dir)

In [ ]:
import os
import re
import base64

print("Starting post-processing of existing final.md files...")

# Iterate through the main folders you processed before
for folder_name in loading_folder:
    folder_path = os.path.join(data_dir, folder_name)
    
    if not os.path.isdir(folder_path):
        continue

    # Iterate through the subdirectories created for each PDF
    for pdf_subfolder_name in os.listdir(folder_path):
        pdf_subfolder_path = os.path.join(folder_path, pdf_subfolder_name)
        
        if not os.path.isdir(pdf_subfolder_path):
            continue
            
        # Define the path to the final.md file
        output_dir = os.path.join(pdf_subfolder_path, "output")
        final_path = os.path.join(output_dir, "final.md")
        print(final_path)

        # Check if final.md exists before trying to process it
        if os.path.exists(final_path):
            
            # --- Core processing logic starts here ---
            
            image_dir = os.path.join(output_dir, "images")
            os.makedirs(image_dir, exist_ok=True)

            with open(final_path, "r", encoding="utf-8") as f:
                content = f.read()

            image_counter = 0
            
            def replace_base64_with_path(match):
                global image_counter
                image_counter += 1
                
                base64_data = match.group(1)
                
                # Fix potential Base64 padding errors
                missing_padding = len(base64_data) % 4
                if missing_padding:
                    base64_data += '=' * (4 - missing_padding)
                    
                try:
                    image_data = base64.b64decode(base64_data)
                    image_filename = f"image_{image_counter}.png"
                    image_save_path = os.path.join(image_dir, image_filename)
                    
                    with open(image_save_path, "wb") as img_file:
                        img_file.write(image_data)
                        
                    relative_image_path = os.path.join("images", image_filename).replace('\\', '/')
                    return f"![]({relative_image_path})"
                except Exception as e:
                    print(f"  - Error decoding/saving image {image_counter} in {final_path}: {e}")
                    return match.group(0) # Return original match on error

            # Regex to find markdown images with Base64 data
            base64_pattern = re.compile(r"!\[.*?\]\(data:image/png;base64,(.*?)\)")
            
            # Perform the replacement
            new_content = base64_pattern.sub(replace_base64_with_path, content)
            
            # Overwrite the final.md with the new content
            with open(final_path, "w", encoding="utf-8") as f:
                f.write(new_content)
            
            if image_counter > 0:
                print(f"Processed {final_path} and extracted {image_counter} images.")
            
            # --- Core processing logic ends here ---

print("\nPost-processing complete!")

In [ ]:
driver.close()